In [1]:
import pandas as pd
import numpy as np

# 2-vote threshold; old split data
vizwiz_train = pd.read_csv('../../data/vizwiz_skill_typ_train.csv', skipinitialspace=True, engine='python')
vizwiz_val = pd.read_csv('../../data/vizwiz_skill_typ_val.csv', skipinitialspace=True, engine='python')
vizwiz_test = pd.read_csv('../../data/vizwiz_skill_typ_test.csv', skipinitialspace=True, engine='python')

vqa_train = pd.read_csv('../../data/vqa_skill_typ_train.csv', skipinitialspace=True, engine='python')
vqa_val = pd.read_csv('../../data/vqa_skill_typ_val.csv', skipinitialspace=True, engine='python')
vqa_test = pd.read_csv('../../data/vqa_skill_typ_test.csv', skipinitialspace=True, engine='python')

vqa_train['QID'] = vqa_train['QID'].astype(str)
vqa_val['QID'] = vqa_val['QID'].astype(str)
vqa_test['QID'] = vqa_test['QID'].astype(str)

# all crowdsourced data
df = pd.read_csv('../../data/VizWiz_VQA_Skills_Data.csv', engine='python')

In [2]:
vqa_train['QID'].dtypes == df['QID'].dtypes == vizwiz_train['QID'].dtypes

True

In [3]:
len(df)

27263

In [4]:
df.columns

Index(['QID', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH', 'SRC_DATASET', 'ANS_TYP',
       'IMG', 'QSN', 'ANS1', 'ANS2', 'ANS3', 'ANS4', 'ANS5', 'ANS6', 'ANS7',
       'ANS8', 'ANS9', 'ANS10', 'NUM_UNIQUE_ANS', 'NUM_MOST_COMMON_ANS'],
      dtype='object')

In [5]:
df['QID'].nunique()

27263

In [6]:
df['SRC_DATASET'].unique()

array(['VQA', 'VizWiz'], dtype=object)

#### cross-reference crowdsourced data to split data

In [7]:
vizwiz_train_questions = vizwiz_train['QID'].unique()
vqa_train_questions = vqa_train['QID'].unique()

vizwiz_val_questions = vizwiz_val['QID'].unique()
vqa_val_questions = vqa_val['QID'].unique()

vizwiz_test_questions = vizwiz_test['QID'].unique()
vqa_test_questions = vqa_test['QID'].unique()

train_questions = list(vizwiz_train_questions) + list(vqa_train_questions)
val_questions = list(vizwiz_val_questions) + list(vqa_val_questions)
test_questions = list(vizwiz_test_questions) + list(vqa_test_questions)

def get_split(qid):
    if qid in train_questions:
        return 'train'
    elif qid in val_questions:
        return 'val'
    elif qid in test_questions:
        return 'test'

# add train/validation/test label to the new dataset so that we can cross-reference
dataset = df.copy()
dataset['split'] = dataset['QID'].apply(get_split)

In [8]:
bad_rows = dataset.loc[dataset['split'].isnull()]
bad_rows

,QID,TXT,OBJ,COL,CNT,OTH,SRC_DATASET,ANS_TYP,IMG,QSN,...,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10,NUM_UNIQUE_ANS,NUM_MOST_COMMON_ANS,split
1889,292432018,3,5,0,2,0,VQA,number,COCO_train2014_000000292432.jpg,What number is on the sign?,...,111,blurry,111,ineligible,115,111,49,8,3,None
2850,39711003,2,3,1,0,0,VQA,yes/no,COCO_train2014_000000039711.jpg,Was there a T in the graffiti?,...,no,no,no,yes,no,yes,no,2,7,None
4949,91360009,0,4,5,0,0,VQA,other,COCO_train2014_000000091360.jpg,What color are the wings of the kite?,...,yellow,yellow,yellow,yellow,orange,yellow,yellow and orange,4,6,None


In [ ]:
# manually fix missing rows (wrong QID)? 
# Currently not performed. don't know if necessary.
"""
bad_rows_idx = list(bad_rows.index)
for i in bad_rows_idx:
    dataset.iloc[i, -1] = 'train'
assert(len(dataset.loc[dataset['split'].isnull()]) == 0)
"""

In [10]:
vqa = dataset.loc[dataset['SRC_DATASET'] == 'VQA']
vizwiz = dataset.loc[dataset['SRC_DATASET'] == 'VizWiz']

In [11]:
# table-1 percentage of VQ for each skill. Change dataset name to switch. 
dset = vqa
(len(dset.loc[dset['OTH'] >= 3]) / len(dset) ) * 100

0.09926543577526306

In [12]:
# none of the above
(len(dset.loc[(dset['OBJ'] < 3) 
           & (dset['COL'] < 3) 
           & (dset['TXT'] < 3)
           & (dset['CNT'] < 3)
           & (dset['OTH'] < 3)]) / len(vizwiz) ) * 100

0.1889678754611716